In [1]:
import pandas as pd

In [2]:
# Define the data
data = {
    'id': [1, 2, 3, 4, 5],
    'distances': [50.5, 120.0, 80.2, 90.1, 150.0]
}

# Create a dataframe
df = pd.DataFrame(data)

df

,id,distances
0,1,50.5
1,2,120.0
2,3,80.2
3,4,90.1
4,5,150.0


In [5]:
from ortools.sat.python import cp_model

# Create a new CpModel
model = cp_model.CpModel()

# Define the variables
x = {}
for id, distance in zip(df['id'], df['distances']):
    x[id] = model.NewBoolVar(f'{id}_feasible')
    model.Add(x[id] == (distance < 100))


In [6]:
# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [7]:
# Print the solution
if status == cp_model.FEASIBLE:
    print('A feasible solution was found')
else:
    print('No feasible solution was found')

No feasible solution was found


In [18]:
import pandas as pd
from ortools.constraint_solver import pywrapcp

# créer le dataframe
data = {'id': [1, 2, 3, 4, 5], 'distances': [50.6, 120.4, 80.2, 90.5, 60.0]}
df = pd.DataFrame(data)

# initialiser le solveur de contraintes
solver = pywrapcp.Solver("test_feasibility")

# créer les variables de décision pour chaque ligne du dataframe
variables = []
for i in range(len(df)):
    variables.append(solver.IntVar(0, 1, f"x{i}"))

# créer la contrainte de distance pour chaque ligne du dataframe
constraints = []
for i in range(len(df)):
    distance = df.loc[i, 'distances']
    if distance <= 100:
        constraints.append(variables[i] == 1)
    else:
        constraints.append(variables[i] == 0)

# ajouter toutes les contraintes au solveur
for constraint in constraints:
    solver.Add(constraint)

# créer une solution
solution = solver.Assignment()
solution.Add(variables)

# créer un solveur de recherche
db = solver.Phase(variables, solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)

# trouver la première solution
solver.NewSearch(db)
if solver.NextSolution():
    print("La contrainte de distance est satisfaisable pour toutes les lignes.")
else:
    print("La contrainte de distance n'est pas satisfaisable pour au moins une ligne.")
solver.EndSearch()


La contrainte de distance est satisfaisable pour toutes les lignes.


In [20]:
import pandas as pd
from ortools.constraint_solver import pywrapcp

# créer le dataframe
data = {'id': [1, 2, 3, 4, 5], 'distances': [50.6, 120.4, 80.2, 90.5, 60.0]}
df = pd.DataFrame(data)

# initialiser le solveur de contraintes
solver = pywrapcp.Solver("test_feasibility")

# créer les variables de décision pour chaque ligne du dataframe
variables = []
for i in range(len(df)):
    variables.append(solver.IntVar(0, 1, f"x{i}"))

# créer la contrainte de distance pour chaque ligne du dataframe
constraints = []
for i in range(len(df)):
    distance = df.loc[i, 'distances']
    if distance > 100:
        constraints.append(variables[i] == 0)
    else:
        constraints.append(variables[i] == 1)

# ajouter toutes les contraintes au solveur
for constraint in constraints:
    solver.Add(constraint)

# créer une solution
# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the solution
if status == cp_model.FEASIBLE:
    print('A feasible solution was found')
else:
    print('No feasible solution was found')

No feasible solution was found


In [25]:
data = {
    'id-parc': [1, 2, 3, 4, 5],
    'superficie-parc': [50.5, 120.0, 80.2, 90.1, 150.0]
}

# Create a dataframe
df = pd.DataFrame(data)

df

,id-parc,superficie-parc
0,1,50.5
1,2,120.0
2,3,80.2
3,4,90.1
4,5,150.0


In [27]:
from ortools.sat.python import cp_model
import math

# Define the maximum number of fountains that can be installed in a park
max_fountains = 3

# Create the CP-SAT model
model = cp_model.CpModel()

# Define the decision variables
num_fountains = {}
for park_id in df['id-parc']:
    num_fountains[park_id] = model.NewIntVar(0, max_fountains, f'num_fountains_{park_id}')

# Define the constraint that limits the number of fountains in each park
for park_id in df['id-parc']:
    model.Add(num_fountains[park_id] <= max_fountains)

# Define the objective function
total_area = math.ceil(sum(df['superficie-parc']))
obj = model.NewIntVar(0, total_area * max_fountains, 'obj')
model.Add(obj == sum(num_fountains[park_id] for park_id in df['id-parc']))

# Define the solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print('Optimal solution found!')
    for park_id in df['id-parc']:
        print(f'Park {park_id} has {solver.Value(num_fountains[park_id])} fountains.')


Optimal solution found!
Park 1 has 0 fountains.
Park 2 has 0 fountains.
Park 3 has 0 fountains.
Park 4 has 0 fountains.
Park 5 has 0 fountains.


In [28]:
from ortools.sat.python import cp_model
import math

# Define the maximum number of fountains that can be installed in a park
max_fountains = 3

# Create the CP-SAT model
model = cp_model.CpModel()

# Define the decision variables
num_fountains = {}
for park_id in df['id-parc']:
    num_fountains[park_id] = model.NewIntVar(0, max_fountains, f'num_fountains_{park_id}')

# Define the constraint that limits the number of fountains in each park
for park_id in df['id-parc']:
    model.Add(num_fountains[park_id] <= max_fountains)

# Define the objective function
total_area = sum(df['superficie-parc'])
obj = model.NewIntVar(0, math.ceil(total_area) * max_fountains, 'obj')
model.Add(obj == sum(num_fountains[park_id] for park_id in df['id-parc']))

# Define the solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print('Optimal solution found!')
    for park_id in df['id-parc']:
        print(f'Park {park_id} has {solver.Value(num_fountains[park_id])} fountains.')


Optimal solution found!
Park 1 has 0 fountains.
Park 2 has 0 fountains.
Park 3 has 0 fountains.
Park 4 has 0 fountains.
Park 5 has 0 fountains.


In [34]:
from ortools.sat.python import cp_model

# Define the data
data = {
    'id-parc': [1, 2, 3, 4, 5],
    'superficie-parc': [50.5, 120.0, 80.2, 90.1, 150.0]
}

# Create a dataframe
df = pd.DataFrame(data)

# Create the CP-SAT model
model = cp_model.CpModel()

# Define the variables
num_parks = len(df)
num_fountains = model.NewIntVar(0, num_parks, 'num_fountains')
fountains_per_park = [model.NewIntVar(1, int(df.loc[i, 'superficie-parc'] / 10), f'fountains_park_{i}') for i in range(num_parks)]

# Define the constraints
model.Add(num_fountains == sum(fountains_per_park))
for i in range(num_parks):
    model.Add(fountains_per_park[i] <= num_fountains)
    
# Define the objective
model.Maximize(num_fountains)

# Create the solver and solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the solution
if status == cp_model.OPTIMAL:
    print(f"Number of fountains: {solver.ObjectiveValue()}")
    for i in range(num_parks):
        print(f"Park {i+1} has {solver.Value(fountains_per_park[i])} fountains.")
else:
    print("No solution found.")


Number of fountains: 5.0
Park 1 has 1 fountains.
Park 2 has 1 fountains.
Park 3 has 1 fountains.
Park 4 has 1 fountains.
Park 5 has 1 fountains.


In [35]:
from ortools.sat.python import cp_model

# Define the data
data = {
    'id-parc': [1, 2, 3, 4, 5],
    'superficie-parc': [50.5, 120.0, 80.2, 90.1, 150.0]
}

# Create a dataframe
df = pd.DataFrame(data)

# Create the CP-SAT model
model = cp_model.CpModel()

# Define the variables
num_parks = len(df)
num_fountains = model.NewIntVar(0, num_parks, 'num_fountains')
fountains_per_park = [model.NewIntVar(1, int(df.loc[i, 'superficie-parc'] / 30), f'fountains_park_{i}') for i in range(num_parks)]

# Define the constraints
model.Add(num_fountains == sum(fountains_per_park))
for i in range(num_parks):
    model.Add(fountains_per_park[i] <= int(df.loc[i, 'superficie-parc'] / 30))
    
# Define the objective
model.Maximize(num_fountains)

# Create the solver and solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the solution
if status == cp_model.OPTIMAL:
    print(f"Number of fountains: {solver.ObjectiveValue()}")
    for i in range(num_parks):
        print(f"Park {i+1} has {solver.Value(fountains_per_park[i])} fountains.")
else:
    print("No solution found.")


Number of fountains: 5.0
Park 1 has 1 fountains.
Park 2 has 1 fountains.
Park 3 has 1 fountains.
Park 4 has 1 fountains.
Park 5 has 1 fountains.


In [36]:
from ortools.sat.python import cp_model
import pandas as pd

# Define the data
data = {
    'id-parc': [1, 2, 3, 4, 5],
    'superficie-parc': [50.5, 120.0, 80.2, 90.1, 150.0]
}

# Create a dataframe
df = pd.DataFrame(data)

# Define the maximum number of fountains for each park
max_fountains = []
for superficie in df['superficie-parc']:
    if superficie > 80:
        max_fountains.append(10)
    else:
        max_fountains.append(5)

# Create the CP-SAT model
model = cp_model.CpModel()

# Define the decision variables
fountains = [model.NewIntVar(0, max_fountains[i], f'fontaines-parc-{i+1}') for i in range(len(df))]

# Define the constraints
for i in range(len(df)):
    model.Add(fountains[i] <= max_fountains[i])

# Define the objective function
model.Maximize(sum(fountains))

# Create the solver and solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the results
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Le nombre total de fontaines maximales est de {solver.ObjectiveValue()}')
    for i in range(len(df)):
        print(f'Le parc {df["id-parc"][i]} a {solver.Value(fountains[i])} fontaines maximales')
else:
    print('Aucune solution trouvée')


Le nombre total de fontaines maximales est de 45.0
Le parc 1 a 5 fontaines maximales
Le parc 2 a 10 fontaines maximales
Le parc 3 a 10 fontaines maximales
Le parc 4 a 10 fontaines maximales
Le parc 5 a 10 fontaines maximales


In [38]:
from ortools.sat.python import cp_model
import pandas as pd

# Define the data
data = {
    'id-parc': [1, 2, 3, 4, 5],
    'superficie-parc': [50.5, 120.0, 80.2, 90.1, 150.0]
}

# Create a dataframe
df = pd.DataFrame(data)

# Define the maximum number of fountains for each park
max_fountains = []
for superficie in df['superficie-parc']:
    if superficie > 80:
        max_fountains.append(10)
    else:
        max_fountains.append(5)

# Create the CP-SAT model
model = cp_model.CpModel()

# Define the decision variables
fountains = [model.NewIntVar(0, max_fountains[i], f'fontaines-parc-{i+1}') for i in range(len(df))]

# Define the constraints
for i in range(len(df)):
    model.Add(fountains[i] <= max_fountains[i])
    # Ajout d'une contrainte pour limiter le nombre de fontaines dans un même parc
    model.Add(fountains[i] <= int(df['superficie-parc'][i] // 10))


# Define the objective function
model.Maximize(sum(fountains))

# Create the solver and solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the results
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Le nombre total de fontaines maximales est de {solver.ObjectiveValue()}')
    for i in range(len(df)):
        print(f'Le parc {df["id-parc"][i]} a {solver.Value(fountains[i])} fontaines maximales')
else:
    print('Aucune solution trouvée')


Le nombre total de fontaines maximales est de 42.0
Le parc 1 a 5 fontaines maximales
Le parc 2 a 10 fontaines maximales
Le parc 3 a 8 fontaines maximales
Le parc 4 a 9 fontaines maximales
Le parc 5 a 10 fontaines maximales


In [39]:
from ortools.sat.python import cp_model
import pandas as pd

# Define the data
data = {
    'id-parc': [1, 2, 3, 4, 5],
    'superficie-parc': [50.5, 120.0, 80.2, 90.1, 150.0],
    'max-fountains': [5, 10, 8, 9, 10]
}

# Create a dataframe
df = pd.DataFrame(data)

# Create the CP-SAT model
model = cp_model.CpModel()

# Define the variables
fountains = [model.NewIntVar(0, df['max-fountains'][i], f'fountains{i}') for i in range(len(df))]

# Define the constraints
for i in range(len(df)):
    # Contrainte pour limiter le nombre de fontaines par parc
    model.Add(fountains[i] <= df['max-fountains'][i])
    # Contrainte pour limiter le nombre de fontaines en fonction de la superficie du parc
    model.Add(fountains[i] <= int(df['superficie-parc'][i] // 10))
    
# Contrainte pour limiter le nombre total de fontaines
model.Add(sum(fountains) <= 40)

# Define the objective function
model.Maximize(sum(fountains))

# Create the solver and solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the solution
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution trouvée avec un nombre total de fontaines de', solver.ObjectiveValue(), ':')
    for i in range(len(df)):
        print(f'Parc {df["id-parc"][i]}: {solver.Value(fountains[i])} fontaines')
else:
    print('Aucune solution trouvée')


Solution trouvée avec un nombre total de fontaines de 40.0 :
Parc 1: 3 fontaines
Parc 2: 10 fontaines
Parc 3: 8 fontaines
Parc 4: 9 fontaines
Parc 5: 10 fontaines
